# Permutation of source time courses for testing

In [30]:
import os
from pathlib import Path
import sys
import mne 
import numpy as np
from mne.datasets import fetch_fsaverage
from tabulate import tabulate
import scipy.io as sio

fs_dir = fetch_fsaverage(verbose=True)
subject = "fsaverage"
subjects_dir = os.path.dirname(fs_dir)
sys.path.append("/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/")
from eeg_toolkit import preprocess, utils
import eeg_toolkit.functional_connectivity as fc
import eeg_toolkit.permutation_testing as pt

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


### Paths

In [31]:
# sl_method = input("Source Localization Method (1- MNE or 2- dSPM): ")
# sl_method = 'MNE' if sl_method == '1' else 'dSPM'
sl_method = 'MNE'

In [32]:
# Settings
times_tup, time_win_path = preprocess.get_time_window(5)

# data paths
data_dir = Path("../../Data")
processed_data_path = data_dir / "Processed Data"
stc_path = data_dir / f"Source Time Courses ({sl_method})"
EO_resting_data_path = stc_path / "Eyes Open"
EC_resting_data_path = stc_path / "Eyes Closed"
zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

# save paths
subset = "FFT Permuted Data"
fc_subset_path = stc_path / subset / "zscored_Epochs" / time_win_path
save_path = fc_subset_path

os.makedirs(save_path, exist_ok=True)

[-2.5,0.0,2.5]


### Get subset of subjects with and without widespread pain

### Manually enter age/sex matched subjects

In [33]:
# All CP and HC subs
sub_ids_CP = [  
    '018',  
    '020',  
    '021',  
    '027',  
    '031',  
    '032',  
    '033',  
    '035',  
    '036',  
    '037',  
    '038',  
    '040',  
    '043',  
    '044',  
    '048',  
    '049',  
    '050',  
    '057',  
    '059',
]

sub_ids_HC = [
    'C10',  
    'C11',  
    'C12',  
    'C13',  
    'C14',  
    'C15',  
    'C16',  
    'C17',  
    'C18',  
    'C19',  
    'C2.',  
    'C24',  
    'C25',  
    'C26',  
    'C27',  
    'C3.',  
    'C6.',  
    'C7.',  
    'C9.'  
]
  
sub_ids = sub_ids_CP + sub_ids_HC

# CP Subs with widespread pain
sub_ids_CP_WSP = [
    '018',
    '022',
    '024',
    '031',
    '032',
    '034',
    '036',
    '039',
    '040',
    '045',
    '046',
    '052',
]

# CP Subs with localized pain
sub_ids_CP_LP = [  
    '020',  
    '021',  
    '023',  
    '029',  
    '031',  
    '037',  
    '041',  
    '042',  
    '048',  
    '049',  
    '050',  
    '056'  
]  

In [34]:
# viable subjects optional import
# sub_ids = utils.import_subs(data_dir, "sub_ids.txt")
# sub_ids = sorted(sub_ids[7:])  # select only 64ch subs and turn into set

# Exclude HEALTHY subjects with widespread pain
widespread_pain_HC = ['C1.','C23']
sub_ids_HC = [sub for sub in sub_ids_HC if sub not in widespread_pain_HC]

In [35]:
print(f"Chronics (all): {len(sub_ids_CP)}")
print([el for el in sub_ids_CP])
print(f"Chronics with widespread pain: {len(sub_ids_CP_WSP)}")
print([el for el in sub_ids_CP_WSP])
print(f"Chronics with localized pain: {len(sub_ids_CP_LP)}")
print([sub for sub in sub_ids_CP_LP])
print(f"Controls: {len(sub_ids_HC)}")
print([el for el in sub_ids_HC])
print(f"\nTotal (Chronics + Controls): {len(sub_ids)}")


Chronics (all): 19
['018', '020', '021', '027', '031', '032', '033', '035', '036', '037', '038', '040', '043', '044', '048', '049', '050', '057', '059']
Chronics with widespread pain: 12
['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052']
Chronics with localized pain: 12
['020', '021', '023', '029', '031', '037', '041', '042', '048', '049', '050', '056']
Controls: 19
['C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']

Total (Chronics + Controls): 38


### Settings for conditions/bands/methods

In [36]:
# Settings
sfreq = 400  # Hz
tmin, tmax = 0.0, 1.25  # exclude the baseline period for connectivity estimation

### Define ROIs, frequency bands, conditions, and methods for FC

In [37]:
####################################################################
# REGIONS OF INTEREST
roi_names = [  # Left
    "rostralanteriorcingulate-lh",  # Left Rostral ACC
    "caudalanteriorcingulate-lh",  # Left Caudal ACC
    "postcentral-lh",  # Left S1,
    "insula-lh",
    "superiorfrontal-lh",  # Left Insula, Left DL-PFC,
    "medialorbitofrontal-lh",  # Left Medial-OFC
    # Right
    "rostralanteriorcingulate-rh",  # Right Rostral ACC
    "caudalanteriorcingulate-rh",  # Right Caudal ACC
    "postcentral-rh",  # , Right S1
    "insula-rh",
    "superiorfrontal-rh",  # Right Insula, Right DL-PFC
    "medialorbitofrontal-rh",
]  # Right Medial-OFC

# Write out ROI names as acronyms
roi_acronyms = ["rACC-lh", "dACC-lh", "S1-lh", "insula-lh", "dlPFC-lh", "mOFC-lh",
                "rACC-rh", "dACC-rh", "S1-rh", "insula-rh", "dlPFC-rh", "mOFC-rh"]
    
####################################################################
# CONDITIONS
conditions = (
    [
        # "Hand 32 mN",
        # 'Hand LS',
        "Hand 256 mN",
        # "Back 32 mN",
        # 'Back LS',
        # "Back 256 mN",
        "Eyes Open",
        # "Eyes Closed",
    ]
)

####################################################################
# CONDITION DICTIONARY
condition_dict = {
        "Hand 32 mN": 0,
        'Hand LS': 1,
        "Hand 256 mN": 2,
        "Back 32 mN": 3,
        'Back LS': 4,
        "Back 256 mN": 5,
        "Eyes Open": 6,
        "Eyes Closed": 7,
}

### Permute source space time signal, repeat 100

In [38]:
this_sub_ids = sub_ids_CP

In [ ]:
repitions = 100

In [43]:
# Loop over each group
for this_sub_ids in [sub_ids_CP, sub_ids_CP_WSP, sub_ids_CP_LP, sub_ids_HC]:
    for sub_id in this_sub_ids:
        # Load the original z-scored epochs data
        label_ts = utils.unpickle_data(zscored_epochs_data_path, f"{sub_id}_epochs.pkl")
        
        # Permute the signals one epoch at a time
        for i,el in enumerate(label_ts):
            print(f"Epoch {i}:",el.shape)
            
            # Create the surrogate data
            surrogate_data = pt.create_surrogate_data(el)
            
            # Replace the original data with the surrogate data
            label_ts[i] = surrogate_data
            
        # Save the surrogate data
        utils.pickle_data(save_path, f"{sub_id}_epochs.pkl", label_ts)

        # Clear display
        utils.clear_display()
        # break

Epoch 0: (12, 2001)
Epoch 1: (12, 2001)
Epoch 2: (12, 2001)
Epoch 3: (12, 2001)
Epoch 4: (12, 2001)
Epoch 5: (12, 2001)
Epoch 6: (12, 2001)
Epoch 7: (12, 2001)
Epoch 8: (12, 2001)
Epoch 9: (12, 2001)
Epoch 10: (12, 2001)
Epoch 11: (12, 2001)
Epoch 12: (12, 2001)
Epoch 13: (12, 2001)
Epoch 14: (12, 2001)
Epoch 15: (12, 2001)
Epoch 16: (12, 2001)
Epoch 17: (12, 2001)
Epoch 18: (12, 2001)
Epoch 19: (12, 2001)
Epoch 20: (12, 2001)
Epoch 21: (12, 2001)
Epoch 22: (12, 2001)
Epoch 23: (12, 2001)
Epoch 24: (12, 2001)
Epoch 25: (12, 2001)
Epoch 26: (12, 2001)
Epoch 27: (12, 2001)
Epoch 28: (12, 2001)
Epoch 29: (12, 2001)
Epoch 30: (12, 2001)
Epoch 31: (12, 2001)
Epoch 32: (12, 2001)
Epoch 33: (12, 2001)
Epoch 34: (12, 2001)
Epoch 35: (12, 2001)
Epoch 36: (12, 2001)
Epoch 37: (12, 2001)
Epoch 38: (12, 2001)
Epoch 39: (12, 2001)
Epoch 40: (12, 2001)
Epoch 41: (12, 2001)
Epoch 42: (12, 2001)
Epoch 43: (12, 2001)
Epoch 44: (12, 2001)
Epoch 45: (12, 2001)
Epoch 46: (12, 2001)
Epoch 47: (12, 2001)
Ep